In [1]:
import re

class Piece:
      
    def __init__(self, position='A1', color='white'):
        self.color = color
        self.position = position.upper()
        self.status = 'man'    
    
    def status_change(self):
        self.status = 'king'
        
    def __str__(self):
        if self.status == 'man':
            return '○' if self.color == 'white' else '●'
        return self.color[0].lower()
    
class CheckerBoard:
    
    @staticmethod
    def indxs2pos(indxs=(0, 0)):
        return f'{chr(indxs[1] + ord("A"))}{indxs[0] + 1}'
    
    @staticmethod
    def position_change(old_position='A1', position='A1'):
        h = ord(position[0].upper()) - ord(old_position[0].upper())
        v = ord(position[1]) - ord(old_position[1])
        return (h, v)
    
    def __init__(self):
        self.paint = [[1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1], 
                      [1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1], 
                      [1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1], 
                      [1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1]]
        
    def __getitem__(self, indxs):
        if isinstance(indxs, str):
            pos = indxs
            indxs = (int(pos[1]) - 1, ord(pos[0].upper()) - ord('A'))
        return self.paint[indxs[0]][indxs[1]]
    
    def __setitem__(self, indxs, value):
        if isinstance(indxs, str):
            pos = indxs
            indxs = (int(pos[1]) - 1, ord(pos[0].upper()) - ord('A'))
        self.paint[indxs[0]][indxs[1]] = value
        
    def __iter__(self):
        for j in range(8):
            yield from [(CheckerBoard.indxs2pos((j, i)), self.paint[j][i]) for i in range(8) if self.paint[j][i] != 0]
        
    def can_move_or_capture(self, piece, new_position='A1'):
        new_position = new_position.upper()
        empty_fields = list(pos for pos, value in self.__iter__() if value == 1)
        if new_position in empty_fields:
            h, v = CheckerBoard.position_change(piece.position, new_position)
            if piece.status == 'man':
                if abs(h) == 1 and v == (1 if piece.color == 'white' else -1) * 1:
                    return True
                if abs(h) == 2 and abs(v) == 2:
                    other_piece_position = f'{chr(ord(piece.position[0]) + h // 2)}{chr(ord(piece.position[1]) + v // 2)}'
                    if isinstance(self[other_piece_position], Piece):
                        if self[other_piece_position].color != piece.color:
                            return other_piece_position
            else:
                if abs(h) == abs(v):
                    counter = 0
                    res = ''
                    for i in range(1, abs(h) + 1):
                        step = f'{chr(ord(piece.position[0]) + h // abs(h) * i)}{chr(ord(piece.position[1]) + v // abs(v) * i)}'           
                        if isinstance(self[step], Piece):
                            if self[step].color != piece.color:
                                counter += 1
                                res = step
                            else:
                                return False
                    if counter > 1:
                        return False
                    if counter == 0:
                        return True
                    return res                    
        return False
    
    def error(self, message=''):
        raise ValueError(f'WRONG  MOVE! {message}')
    
    def put_piece(self, old_position=None, new_position=None, new_piece=None):
        if isinstance(new_piece, Piece):
            pos = new_piece.position
            self.paint[int(pos[1]) - 1][ord(pos[0].upper()) - ord('A')] = new_piece
            return None
        piece = self[old_position]
        if isinstance(piece, Piece):
            if not new_position is None:
                move = self.can_move_or_capture(piece, new_position)
                if isinstance(move, str):
                    pos = piece.position
                    piece.position = new_position
                    self[new_position] = piece
                    self[move] = 1
                    self[pos] = 1
                elif move:
                    pos = piece.position
                    piece.position = new_position
                    self[new_position] = piece
                    self[pos] = 1
                else:
                    self.error(f"can't move from {old_position} to {new_position}")
                    return None
                
                if (new_position[1], piece.color) == ('1', 'black') or (new_position[1], piece.color) == ('8', 'white'):
                    piece.status_change()
                
        else:
            self.error('empty cell')
        
    def print_board(self):
        for i in reversed(range(8)):
            line = []
            for q in self.paint[i]:
                if q == 1:
                    line.append('▪')
                elif q == 0:
                    line.append('▫')
                else:
                    line.append(q)
            print((i + 1), ' ', *line)
        print()
        print(*list('  ABCDEFGH'))

class Game:
    
    def __init__(self):
        game_board = CheckerBoard()
        pieces = list(iter(game_board))
        pieces_num = len(pieces)
        for q in range(12):
            white_piece = Piece(pieces[q][0])
            game_board.put_piece(new_piece=white_piece)
        for q in range(12):
            black_piece = Piece(pieces[pieces_num - q - 1][0], color='black')
            game_board.put_piece(new_piece=black_piece)
        self.game_board = game_board
        self.player = 'white'
        
    def error(self, message=''):
        raise ValueError(f"WRONG MOVE! {message}")
        
    def player_change(self):
        new_color = 'black' if self.player == 'white' else 'white'
        self.player = new_color
    
    def player_input(self, obj=None):
        move_format = r'[a-hA-H][1-8]( [a-hA-H][1-8])+'
        if obj is None:
            res = input()
        else:
            res = ' '.join(f'{c}{r}' for c, r in zip(obj.input_col, obj.input_row))
        while re.fullmatch(move_format, res) is None:
            self.error('wrong move format')
            if obj is None:
                res = input()
            else:
                res = ' '.join(f'{c}{r}' for c, r in zip(obj.input_col, obj.input_row))
        return res.split()
        
    def move(self, move_positions=None, obj=None):
        if move_positions is None:
            positions = self.player_input(obj)
        else:
            positions = move_positions
        if len(positions) == 2:
            old_position, new_position = positions
            if not isinstance(self.game_board[old_position], Piece):
                self.error('empty cell')
                self.move(obj=obj)
                return None
            if self.game_board[old_position].color != self.player:
                self.error(f"{self.player}'s move")
                self.move(obj=obj)
                return None
            try:
                self.game_board.put_piece(old_position, new_position)
            except ValueError as e:
                self.error(e)
                self.move(obj=obj)
                return None
        else:
            for new_move_positions in zip(positions, positions[1:]):
                piece_counter = self.game_status()['black' if self.player == 'white' else 'white']
                self.move(new_move_positions)
                self.player_change()
                if piece_counter == self.game_status()['black' if self.player == 'white' else 'white']:
                    break
        self.player_change()
        if self.is_blocked():
            self.player_change()
            self.win(obj)
            return 0
        
    def if_can_capture(self, position):
        piece = [value for p, value in self.game_board.__iter__() if p == position][0]
        empty_fields = list(pos for pos, value in self.game_board.__iter__() if value == 1)
        flag = False
        for pos in empty_fields:
            if isinstance(self.game_board.can_move_or_capture(piece, pos), str):
                flag = True
                break
        return flag
        
    def game_status(self):
        pieces = {'white' : 0, 'black' : 0}
        for pos, value in self.game_board.__iter__():
            if isinstance(value, Piece):
                pieces[value.color] += 1
        return pieces
    
    def is_blocked(self):
        if self.game_status()[self.player] == 0:
            return True
        pieces = (value for pos, value in self.game_board.__iter__() if isinstance(value, Piece) and value.color == self.player)
        empty_fields = list(pos for pos, value in self.game_board.__iter__() if value == 1)
        flag = True
        for piece in pieces:
            for pos in empty_fields:
                if not self.game_board.can_move_or_capture(piece, pos) is False:
                    flag = False
                    break
        return flag
    
    def win(self, obj=None):
        if obj is None:
            print(f'{self.player.title()} won! Congratulations!!!')
        else:
            obj.messanges = f'{self.player.title()} won! Congratulations!!!'
        
    def start(self):
        while self.move() is None:
            self.game_board.print_board()
            
            
            
class TestGame(Game):
    
    def __init__(self, filename):
        game_board = CheckerBoard()
        with open(filename, 'r', encoding='utf-8') as file:
            for line in file.readlines():
                text = line.strip()
                if text == 'stop':
                    break
                pos, c = text.split()
                color = 'black' if c.lower() == 'b' else 'white'
                new_piece = Piece(pos, color)
                if c.isupper():
                    new_piece.status_change()
                game_board.put_piece(new_piece=new_piece)
        self.game_board = game_board
        self.player = 'white'

    def start(self):
        while self.move() is None:
            print('a')

In [2]:
import sys
from PyQt5.QtWidgets import QWidget, QToolTip, QPushButton, QApplication, QDesktopWidget, QLabel 
from PyQt5.QtWidgets import QMainWindow, qApp, QAction #, QMessageBox
from PyQt5.QtCore import QCoreApplication, QLine
from PyQt5.QtGui import QFont, QIcon, QPainter, QColor, QBrush, QPen

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

class Example(QMainWindow):

    def __init__(self):
        super().__init__()
        self.input_row = []
        self.input_col = []
        self.can_end_move = False
        self.messanges = None
        self.game = Game()
        self.initUI()

    def text(self, text, x, y):
        lbl = QLabel(text, self)
        lbl.move(x, y)
        
    def paintEvent(self, e):
        qp = QPainter()
        qp.begin(self)
        x0, y0 = 80, 100
        size = 60
        self.draw_empty_board(qp, x0, y0, size)
        self.draw_board(qp, self.game.game_board, x0, y0, size)
        #qp.end()
        
    def draw_circle(self, qp, c, x, y, radius, width):
        qp.setBrush(QColor(c, c, c))
        qp.drawEllipse(x - (radius - width) // 2, y - (radius - width) // 2, radius - width, radius - width)
        
    def draw_piece(self, qp, x, y, color='white', status='king', radius=45):
        if color == 'white':
            c = 255
            qp.setPen(QPen(QColor(c, c, c), 0))
            self.draw_circle(qp, c, x, y, radius, width=0)
            if status == 'man':
                self.draw_circle(qp, 255 - c, x, y, radius, width=12)
                self.draw_circle(qp, c, x, y, radius, width=20)    
            else:
                self.draw_crown(qp, 255 - c, x, y + radius / 4)
        else:
            c = 0
            qp.setPen(QPen(QColor(c, c, c), 0))
            self.draw_circle(qp, 255 - c, x, y, radius, width=-4)
            self.draw_circle(qp, c, x, y, radius, width=4)
            if status == 'man':
                self.draw_circle(qp, 255 - c, x, y, radius, width=12)
                self.draw_circle(qp, c, x, y, radius, width=20)
            else:
                self.draw_crown(qp, 255 - c, x, y + radius / 4)
           
    def draw_crown(self, qp, c, x00, y00, size=4):
        qp.setPen(QPen(QColor(c, c, c), 2))
        points = ((0, 0), (4, 0), (6, 4), (4, 3), (4, 6), (2, 4), (0, 7))  
        x0, y0 = (0, 0)
        for x, y in points[1:]:
            qp.drawLine(QLine(x00 + size * x0, y00 - size * y0, x00 + size * x, y00 - size * y))
            x0, y0 = x, y      
        for x, y in reversed(points):
            qp.drawLine(QLine(x00 - size * x0, y00 - size * y0, x00 - size * x, y00 - size * y))
            x0, y0 = x, y
        
    def draw_empty_board(self, qp, x0=0, y0=0, size=50):
        col = QColor(0, 0, 0)
        qp.setPen(col)
        
        for row in range(8):
            for col in range(8):
                color = 255 if col % 2 == row % 2 else 0
                qp.setBrush(QColor(color, color, color))
                qp.drawRect(x0 + col * size, y0 + row * size, size, size)
                
    def draw_board(self, qp, board, x0=0, y0=0, size=50):
        def from_pos_to_xy(pos, x0, y0, size):
            x = x0 + size / 2 + (ord(pos[0].upper()) - ord('A')) * size     
            y = y0 + size / 2 + (8 - int(pos[1])) * size
            return x, y
        
        col = QColor(0, 0, 0)
        qp.setPen(col)
                
        for pos, piece in board:
            if isinstance(piece, Piece):
                x, y = from_pos_to_xy(pos, x0, y0, size)
                self.draw_piece(qp, x, y, piece.color, piece.status, size - 4)
                
        
    def center(self, width=800, hight=800):
        self.resize(width, hight)
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
    

                
    def button(self, name, x=450, y=450, sizex=None, sizey=None):
        QToolTip.setFont(QFont('SansSerif', 10))
        btn = QPushButton(name, self)
        btn.setToolTip(f'This is {"column" if name in "ABCDEFGH" else "row"} <b>{name}</b>')
        if sizex is None and sizey is None:
            btn.resize(btn.sizeHint())
        else:
            btn.resize(sizex, sizey)
        btn.move(x, y)
        
        btn.clicked.connect(lambda: self.try_something(name))
        
    def restart(self):
        self.game = Game()
        self.update()
        self.statusBar().showMessage(f"now is {self.game.player}'s move")
        
    def restart_move(self):
        self.input_row = []
        self.input_col = []
    
    def try_something(self, name):
        if name in "ABCDEFGH":
            self.input_col.append(name)
        else:
            self.input_row.append(name)  
        if len(self.input_col) == len(self.input_row) == 2:
            try:
                start = self.game.game_status()
                self.game.move(obj=self)
                self.update()
                self.can_end_move = False
                if self.messanges is None:
                    can_capture = self.game.if_can_capture(f'{self.input_col[-1]}{self.input_row[-1]}')
                    if start != self.game.game_status() and can_capture:
                        self.can_end_move = True
                        self.game.player_change()
                        self.statusBar().showMessage(f"It is still{self.game.player}'s move: select one cell to move to or press Ctrl+X to end move")
                        del self.input_row[0]
                        del self.input_col[0]
                    else: 
                        self.statusBar().showMessage(f"now is {self.game.player}'s move")
                        self.input_row = []
                        self.input_col = []
                else:
                    self.statusBar().showMessage(self.messanges)
            except Exception as err:
                self.statusBar().showMessage(f'{err}')
                self.input_row = []
                self.input_col = []
    
    
    def end_move(self):
        if self.can_end_move:
            self.game.player_change()
            self.input_row = []
            self.input_col = []
            self.can_end_move = False
        else:
            if not len(self.input_col) == len(self.input_row) == 2:
                self.input_row = []
                self.input_col = []
                self.statusBar().showMessage(f"It is still {self.game.player}'s move: reseted")
            else:
                self.statusBar().showMessage(f'Make at least 1 movement')
        
    def add_menubar(self):
        restart = QAction(QIcon('restart.png'), '&Restart game', self)
        restart.setShortcut('Ctrl+Z')
        restart.setStatusTip('Start completely new game')
        restart.triggered.connect(self.restart)
        
        end_move = QAction(QIcon('end_move.png'), '&End move', self)
        end_move.setShortcut('Ctrl+X')
        end_move.setStatusTip('End move or reset selected fields (in case of mistake reseted automatically)')
        end_move.triggered.connect(self.end_move)
        
        menubar = self.menuBar()
        Menu = menubar.addMenu('&Menu')
        Menu.addAction(end_move)
        Menu.addAction(restart)
        
        
    def initUI(self):
        size = 60
        x0, y0 = 80, 100
        line = 10
        self.center(2 * line + x0 + 9 * size, 4 * line + y0 + 9 * size)
        self.setWindowTitle('Checkers')
        self.setWindowIcon(QIcon('icon.png'))

        
        for letter in reversed(list('12345678')):
            x, y = line, y0 - size + (1 + abs(ord(letter) - ord('8'))) * size
            self.button(letter, x, y, sizex=size, sizey=size) 
            x, y = 3 * line + 9 * size, y0 - size + (1 + abs(ord(letter) - ord('8'))) * size
            self.button(letter, x, y, sizex=size, sizey=size) 
        for letter in 'ABCDEFGH':
            x, y = 2 * line + (1 + ord(letter) - ord('A')) * size, y0 - size - line
            self.button(letter, x, y, sizex=size, sizey=size)
            x, y = 2 * line + (1 + ord(letter) - ord('A')) * size, y0 - size + line + 9 * size 
            self.button(letter, x, y, sizex=size, sizey=size)   
            
        self.statusBar().showMessage(f"now is {self.game.player}'s move")
        self.add_menubar()
        self.show()


if __name__ == '__main__':

    app = QApplication(sys.argv)
    ex = Example()    
    
    x0, y0 = 80, 100
    size = 60
    ex.game = TestGame('game1.txt') 
    
    sys.exit(app.exec_())
    
    

SystemExit: 0